In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import os
from dotenv import load_dotenv
from agents import OpenAIChatCompletionsModel,AsyncOpenAI
import logfire
load_dotenv()
try:
        
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
    if not GEMINI_API_KEY:
        raise ValueError("GEMINI_API_KEY is not defined in .env or not imported successfully")
    client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",api_key=GEMINI_API_KEY)
    logfire_token=os.getenv("LOGFIR_TOKEN") 
    # logfire.configure(token=logfire_token)
    model = OpenAIChatCompletionsModel(model='gemini-1.5-flash',openai_client=client)
    print(client)
    print(model)
except Exception as e:
    print(f"Error",e)
    

In [23]:
from dataclasses import dataclass
from agents import Agent,Runner, Usage,set_tracing_disabled,RunContextWrapper,tool
import asyncio

@dataclass
class MyContext:
    role:str
    country:str
@dataclass
class TokenCount:
    input_token:int
    output_token:int
    total_token:int
        
 
def dynamic_instruction(ctx:RunContextWrapper,agent:Agent[MyContext]):
    print("📊 Usage inside dynamic_instruction:")
    print("  Input tokens:", ctx.usage.input_tokens)
    print("  Output tokens:", ctx.usage.output_tokens)
    print("  Total tokens:", ctx.usage.total_tokens)
    return f"You are an helpful assistant . your role is {ctx.context.role} and use the national language of this country {ctx.context.country}and use the  output_tokens like {ctx.usage.output_tokens}"
async def main(input_value:str):
    
  
    set_tracing_disabled(disabled=True) 
    agent = Agent(
        name='basic_chatbot', 
        instructions=lambda ctx: dynamic_instruction(ctx, agent=None) + "\n\nThen call the tool `show_usage`.",
        model=model,
        
    )
    our_token = TokenCount(input_token=100,output_token=200,total_token=400)
    our_context = MyContext('PYthon developer','USA') 
    resp =await Runner.run(starting_agent=agent,input=input_value,context=our_context,usage=our_token)
    print(resp.final_output)
 

if __name__ == "__main__":
    asyncio.run(main('What is Data Science'))
        
    


TypeError: Runner.run() got an unexpected keyword argument 'usage'

In [26]:
from dataclasses import dataclass
from agents import Agent, Runner, set_tracing_disabled, RunContextWrapper, function_tool
import asyncio

@dataclass
class MyContext:
    role: str
    country: str

# Tool to show real usage after LLM runs
@function_tool
def show_usage(ctx: RunContextWrapper[MyContext]) -> str:
    resp = (
        f"🔢 Token usage:\n"
        f"- Input: {ctx.usage.input_tokens}\n"
        f"- Output: {ctx.usage.output_tokens}\n"
        f"- Total: {ctx.usage.total_tokens}"
    )
    print(resp)
    return resp

# Dynamic system prompt
def dynamic_instruction(ctx: RunContextWrapper[MyContext], agent: Agent[MyContext]) -> str:
    return (
        f"You are a helpful assistant. Your role is {ctx.context.role}, "
        f"and the national language of this country ({ctx.context.country}) should be used."
    )

# Async main function
async def main(input_value: str):
    set_tracing_disabled(True)

    agent = Agent[MyContext](
        name="basic_chatbot",
        instructions=dynamic_instruction,
        model=model,
        tools=[show_usage],  # Important
    )

    context = MyContext(role="Python developer", country="USA")
    # Trigger show_usage tool after answer
    response = await Runner.run(
        starting_agent=agent,
        input=input_value + "\n\nThen call the tool `show_usage`.",
        context=context
    )

    print("🤖 Final Output:\n", response.final_output)

if __name__ == "__main__":
    asyncio.run(main("What is Data Science?"))


🔢 Token usage:
- Input: 44
- Output: 78
- Total: 122
🤖 Final Output:
 The `show_usage` function doesn't appear to actually *show* usage;  it's likely meant to report token usage (input and output tokens).  My previous response included the explanation of Data Science, followed by the call to `show_usage`, and the output indicating token counts related to that interaction.

